In [1]:
import pandas as pd 
import numpy as np 
import matplotlib as plt
import requests
from io import StringIO
from google.colab import drive
import random

from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
%pip install statsmodels
import statsmodels.api as sm



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


# Predicting New York Yellow Cab Prices

Team Members: Bhuvika Jaiswal, Carter Neumann, Lavanaya Raju, Natasha Raut, Faisal Qureshi 

## Data Collection and Processing

In [3]:
p = 0.001  # 0.1% of the data as original table consists of 30.9 million rows
taxi_2021 = pd.read_csv("/content/drive/MyDrive/INDENG142/2021_Yellow_Taxi_Trip_Data.csv",  skiprows=lambda i: i>0 and random.random() > p)

In [4]:
taxi_2021 = taxi_2021[taxi_2021['passenger_count'].notna()]
taxi_2021 = taxi_2021[taxi_2021['fare_amount'] > 0]

In [5]:
taxi_2021

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,2.0,01/01/2021 12:19:18 AM,01/01/2021 12:39:16 AM,4.0,5.04,1.0,N,137,49,1.0,18.5,0.5,0.5,4.00,0.0,0.3,26.30,2.5
1,1.0,01/01/2021 12:32:30 AM,01/01/2021 12:39:56 AM,1.0,1.50,1.0,N,164,107,1.0,7.5,3.0,0.5,2.80,0.0,0.3,14.10,2.5
2,2.0,01/01/2021 12:31:41 AM,01/01/2021 12:47:34 AM,1.0,5.67,1.0,N,161,88,2.0,19.0,0.5,0.5,0.00,0.0,0.3,22.80,2.5
3,2.0,01/01/2021 01:43:59 AM,01/01/2021 01:49:36 AM,1.0,1.02,1.0,N,74,41,2.0,5.5,0.5,0.5,0.00,0.0,0.3,6.80,0.0
4,2.0,01/01/2021 01:25:27 AM,01/01/2021 01:35:08 AM,1.0,1.03,1.0,N,48,161,1.0,7.5,0.5,0.5,2.26,0.0,0.3,13.56,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31046,1.0,12/31/2021 10:06:23 PM,12/31/2021 10:33:09 PM,2.0,4.40,1.0,N,163,231,1.0,19.5,3.0,0.5,4.65,0.0,0.3,27.95,2.5
31047,1.0,12/31/2021 10:53:31 PM,12/31/2021 11:03:19 PM,1.0,1.60,1.0,N,50,158,1.0,7.5,3.0,0.5,3.40,0.0,0.3,14.70,2.5
31048,1.0,12/31/2021 10:42:19 PM,12/31/2021 10:54:51 PM,1.0,2.70,1.0,N,41,163,1.0,11.5,3.0,0.5,3.00,0.0,0.3,18.30,2.5
31049,2.0,12/31/2021 11:29:11 PM,12/31/2021 11:38:03 PM,1.0,2.98,1.0,N,142,166,1.0,10.5,0.5,0.5,2.86,0.0,0.3,17.16,2.5


In [6]:
taxi_2021['tpep_pickup_datetime'] = pd.to_datetime(taxi_2021['tpep_pickup_datetime'])
taxi_2021['tpep_dropoff_datetime'] = pd.to_datetime(taxi_2021['tpep_dropoff_datetime'])
taxi_2021['trip_time'] = taxi_2021['tpep_dropoff_datetime'].sub(taxi_2021['tpep_pickup_datetime']).view('int64') // (10**9)
taxi_2021

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,trip_time
0,2.0,2021-01-01 00:19:18,2021-01-01 00:39:16,4.0,5.04,1.0,N,137,49,1.0,18.5,0.5,0.5,4.00,0.0,0.3,26.30,2.5,1198
1,1.0,2021-01-01 00:32:30,2021-01-01 00:39:56,1.0,1.50,1.0,N,164,107,1.0,7.5,3.0,0.5,2.80,0.0,0.3,14.10,2.5,446
2,2.0,2021-01-01 00:31:41,2021-01-01 00:47:34,1.0,5.67,1.0,N,161,88,2.0,19.0,0.5,0.5,0.00,0.0,0.3,22.80,2.5,953
3,2.0,2021-01-01 01:43:59,2021-01-01 01:49:36,1.0,1.02,1.0,N,74,41,2.0,5.5,0.5,0.5,0.00,0.0,0.3,6.80,0.0,337
4,2.0,2021-01-01 01:25:27,2021-01-01 01:35:08,1.0,1.03,1.0,N,48,161,1.0,7.5,0.5,0.5,2.26,0.0,0.3,13.56,2.5,581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31046,1.0,2021-12-31 22:06:23,2021-12-31 22:33:09,2.0,4.40,1.0,N,163,231,1.0,19.5,3.0,0.5,4.65,0.0,0.3,27.95,2.5,1606
31047,1.0,2021-12-31 22:53:31,2021-12-31 23:03:19,1.0,1.60,1.0,N,50,158,1.0,7.5,3.0,0.5,3.40,0.0,0.3,14.70,2.5,588
31048,1.0,2021-12-31 22:42:19,2021-12-31 22:54:51,1.0,2.70,1.0,N,41,163,1.0,11.5,3.0,0.5,3.00,0.0,0.3,18.30,2.5,752
31049,2.0,2021-12-31 23:29:11,2021-12-31 23:38:03,1.0,2.98,1.0,N,142,166,1.0,10.5,0.5,0.5,2.86,0.0,0.3,17.16,2.5,532


In [7]:
jan_fht = (pd.read_parquet("/content/drive/MyDrive/INDENG142/fhvhv_tripdata_2021-01.parquet")).sample(n=2500)



In [8]:
feb_fht = pd.read_parquet("/content/drive/MyDrive/INDENG142/fhvhv_tripdata_2021-02.parquet").sample(n=2500)
mar_fht = pd.read_parquet("/content/drive/MyDrive/INDENG142/fhvhv_tripdata_2021-03.parquet").sample(n=2500)
apr_fht = pd.read_parquet("/content/drive/MyDrive/INDENG142/fhvhv_tripdata_2021-04.parquet").sample(n=2500)
may_fht = pd.read_parquet("/content/drive/MyDrive/INDENG142/fhvhv_tripdata_2021-05.parquet").sample(n=2500)
jun_fht = pd.read_parquet("/content/drive/MyDrive/INDENG142/fhvhv_tripdata_2021-06.parquet").sample(n=2500)
jul_fht = pd.read_parquet("/content/drive/MyDrive/INDENG142/fhvhv_tripdata_2021-07.parquet").sample(n=2500)
aug_fht = pd.read_parquet("/content/drive/MyDrive/INDENG142/fhvhv_tripdata_2021-08.parquet").sample(n=2500)
sep_fht = pd.read_parquet("/content/drive/MyDrive/INDENG142/fhvhv_tripdata_2021-09.parquet").sample(n=2500)
oct_fht = pd.read_parquet("/content/drive/MyDrive/INDENG142/fhvhv_tripdata_2021-10.parquet").sample(n=2500)
nov_fht = pd.read_parquet("/content/drive/MyDrive/INDENG142/fhvhv_tripdata_2021-11.parquet").sample(n=2500)
dec_fht = pd.read_parquet("/content/drive/MyDrive/INDENG142/fhvhv_tripdata_2021-12.parquet").sample(n=2500)

In [9]:
frames = [jan_fht, feb_fht, mar_fht, apr_fht,may_fht,jun_fht, jul_fht,aug_fht, sep_fht, oct_fht,  nov_fht, dec_fht  ]
fht_2021 = pd.concat(frames)
fht_2021[['hvfhs_license_num']== 'HV0002']


,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
1848924,HV0003,B02835,B02835,2021-01-06 13:17:36,2021-01-06 13:19:38,2021-01-06 13:19:50,2021-01-06 13:36:51,238,243,5.670,...,1.49,2.75,NaN,0.0,14.80,N,N,,N,N
2734191,HV0003,B02882,B02882,2021-01-08 19:31:34,2021-01-08 19:38:32,2021-01-08 19:38:39,2021-01-08 20:00:18,182,94,4.370,...,1.36,0.00,NaN,0.0,15.69,N,N,,N,N
5109701,HV0003,B02877,B02877,2021-01-15 07:37:43,2021-01-15 07:42:35,2021-01-15 07:44:37,2021-01-15 07:49:58,94,136,1.000,...,0.64,0.00,NaN,0.0,6.55,N,N,,N,N
2232566,HV0005,B02510,None,2021-01-07 14:08:25,NaT,2021-01-07 14:13:19,2021-01-07 14:26:48,165,26,2.050,...,1.17,0.00,NaN,0.0,9.04,N,N,N,N,N
5522108,HV0003,B02879,B02879,2021-01-16 01:56:39,2021-01-16 01:57:17,2021-01-16 01:58:27,2021-01-16 02:14:30,244,70,8.820,...,2.24,0.00,NaN,0.0,17.79,N,N,,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8371964,HV0005,B03406,None,2021-12-15 14:41:11,NaT,2021-12-15 15:00:56,2021-12-15 15:21:09,169,159,2.319,...,1.51,0.00,0.0,0.0,13.14,N,N,N,N,N
4831182,HV0003,B03404,B03404,2021-12-09 15:47:49,2021-12-09 15:51:20,2021-12-09 15:53:22,2021-12-09 16:34:04,82,216,6.680,...,2.54,0.00,0.0,0.0,39.41,N,N,,N,Y
4767514,HV0003,B03404,B03404,2021-12-09 13:11:57,2021-12-09 13:13:23,2021-12-09 13:15:18,2021-12-09 13:42:37,142,231,5.460,...,2.83,2.75,0.0,0.0,23.96,N,N,,N,N
4013433,HV0003,B03404,B03404,2021-12-08 06:09:42,2021-12-08 06:12:18,2021-12-08 06:12:54,2021-12-08 07:05:53,213,97,17.300,...,3.98,0.00,0.0,0.0,47.05,N,N,,N,N


In [ ]:
fht_2021['airport_fee'] = fht_2021['airport_fee'].fillna(0)
fht_2021 = fht_2021.drop(columns=['dispatching_base_num', 'originating_base_num', 'on_scene_datetime'])


In [ ]:
fht_2021['total_fare'] = fht_2021['base_passenger_fare'] + fht_2021['tolls'] + fht_2021['bcf'] + fht_2021['tips'] + fht_2021['sales_tax'] + fht_2021['airport_fee'] + fht_2021['congestion_surcharge']
dict = {"HV0002" : 'Juno', "HV0003" : 'Uber', "HV0004": 'Via', "HV0005" : 'Lyft'}
fht_2021=fht_2021.replace({"hvfhs_license_num": dict})
fht_2021.rename(columns={'hvfhs_license_num': 'fht_company'}, inplace=True)
pd.get_dummies(fht_2021.fht_company)
company_dummies = pd.get_dummies(fht_2021.fht_company)
fht_2021 = pd.concat([fht_2021, company_dummies], axis=1)
fht_2021


,fht_company,request_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,...,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag,total_fare,Lyft,Uber,Via
11524553,Uber,2021-01-31 00:14:52,2021-01-31 00:20:49,2021-01-31 00:31:47,255,97,3.000,658,13.36,0.00,...,8.82,N,N,,N,N,14.95,0,1,0
2108504,Lyft,2021-01-07 07:17:43,2021-01-07 07:21:43,2021-01-07 07:56:51,165,90,10.679,2108,40.98,5.74,...,29.51,N,N,N,N,N,55.02,1,0,0
1301799,Uber,2021-01-04 19:11:02,2021-01-04 19:15:46,2021-01-04 19:27:36,10,132,4.530,710,9.84,0.00,...,10.95,N,N,,N,N,11.01,0,1,0
7753307,Uber,2021-01-22 03:02:18,2021-01-22 03:10:04,2021-01-22 03:39:09,132,185,19.060,1746,60.11,6.12,...,43.56,N,N,,N,N,74.10,0,1,0
5915242,Uber,2021-01-16 23:27:30,2021-01-16 23:35:30,2021-01-16 23:46:13,95,82,1.610,643,7.58,0.00,...,7.16,N,N,,N,N,10.48,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13254587,Uber,2021-12-25 00:22:58,2021-12-25 00:35:05,2021-12-25 00:52:34,185,20,2.310,1049,13.33,0.00,...,12.28,N,N,,N,N,14.91,0,1,0
487807,Uber,2021-12-01 23:04:48,2021-12-01 23:07:04,2021-12-01 23:11:38,17,217,0.580,274,7.91,0.00,...,5.76,N,N,,N,N,8.85,0,1,0
13885145,Uber,2021-12-26 18:24:03,2021-12-26 18:28:09,2021-12-26 18:50:22,71,61,2.680,1333,15.62,0.00,...,14.87,N,N,,N,N,17.48,0,1,0
6098470,Uber,2021-12-11 14:13:51,2021-12-11 14:18:56,2021-12-11 14:35:07,226,138,5.480,971,17.89,0.00,...,15.27,N,N,,N,N,27.37,0,1,0


In [ ]:
fht_2021.describe()

,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,total_fare,Lyft,Uber,Via
count,30000.000000,30000.00000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.00000,3.000000e+04,30000.000000,30000.000000,30000.000000
mean,137.009900,140.93510,4.818434,1092.152367,21.809288,0.920761,0.677924,1.866155,1.044508,0.112500,0.824652,17.52953,2.725579e+01,0.275300,0.719567,0.005133
std,75.685975,78.18365,5.508019,764.200247,17.217413,3.416313,0.579401,1.466600,1.346414,0.522273,2.596714,13.68775,2.208323e+01,0.446673,0.449218,0.071464
min,3.000000,1.00000,0.000000,1.000000,-26.580000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.03000,-4.440892e-16,0.000000,0.000000,0.000000
25%,72.000000,74.00000,1.600000,568.000000,10.780000,0.000000,0.320000,0.910000,0.000000,0.000000,0.000000,8.61000,1.323000e+01,0.000000,0.000000,0.000000
50%,138.000000,141.00000,2.980000,896.000000,17.100000,0.000000,0.510000,1.460000,0.000000,0.000000,0.000000,13.75500,2.089000e+01,0.000000,1.000000,0.000000
75%,210.000000,216.00000,5.933250,1390.000000,26.910000,0.000000,0.830000,2.350000,2.750000,0.000000,0.000000,22.10000,3.355000e+01,1.000000,1.000000,0.000000
max,265.000000,265.00000,123.135000,12812.000000,376.490000,44.900000,11.480000,33.960000,5.500000,5.000000,63.540000,289.01000,4.308000e+02,1.000000,1.000000,1.000000


In [ ]:
fht_2021.isna().sum()


fht_company             0
request_datetime        0
pickup_datetime         0
dropoff_datetime        0
PULocationID            0
DOLocationID            0
trip_miles              0
trip_time               0
base_passenger_fare     0
tolls                   0
bcf                     0
sales_tax               0
congestion_surcharge    0
airport_fee             0
tips                    0
driver_pay              0
shared_request_flag     0
shared_match_flag       0
access_a_ride_flag      0
wav_request_flag        0
wav_match_flag          0
total_fare              0
Lyft                    0
Uber                    0
Via                     0
dtype: int64

**ANALYTICS MODELS**

In [ ]:
##one way to add additional data for comparative study is to compare yellow cab taxi prices with uber ??? Uber vs Yellow Taxis 

In [ ]:
train, test = train_test_split(taxi_2021,test_size=0.3, random_state = 42)

Calculating VIF to check for multi-collinearity

In [ ]:
# function to calculate VIF!

from statsmodels.stats.outliers_influence import variance_inflation_factor

def VIF(df, columns):
    values = sm.add_constant(df[columns]).values
    num_columns = len(columns)+1
    vif = [variance_inflation_factor(values, i) for i in range(num_columns)]
    return pd.Series(vif[1:], index=columns)

In [ ]:
# turn Y and N into binary
train["store_and_fwd_flag"] = train["store_and_fwd_flag"].replace({"Y":1, "N":0})
test["store_and_fwd_flag"] = test["store_and_fwd_flag"].replace({"Y":1, "N":0})

In [ ]:
# extracting columns as a list
col_list = list(train.columns)
col_list.remove("total_amount")

# converting datetime to julian datetime
train["tpep_pickup_datetime"] = train["tpep_pickup_datetime"].apply(lambda x: x.to_julian_date())
train["tpep_dropoff_datetime"] = train["tpep_dropoff_datetime"].apply(lambda x: x.to_julian_date())

test["tpep_pickup_datetime"] = test["tpep_pickup_datetime"].apply(lambda x: x.to_julian_date())
test["tpep_dropoff_datetime"] = test["tpep_dropoff_datetime"].apply(lambda x: x.to_julian_date())

# VIF score for all columns
VIF((train), col_list)

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


VendorID                 4.911823e+00
tpep_pickup_datetime              inf
tpep_dropoff_datetime             inf
passenger_count          1.023150e+00
trip_distance            2.040065e+00
RatecodeID               1.038791e+00
store_and_fwd_flag       1.029657e+00
PULocationID             1.033992e+00
DOLocationID             1.030029e+00
payment_type             1.463562e+00
fare_amount              1.010319e+00
extra                    4.975927e+00
mta_tax                  1.194008e+00
tip_amount               2.025603e+00
tolls_amount             1.662999e+00
improvement_surcharge    1.036199e+00
congestion_surcharge     1.325501e+00
trip_time                1.501200e+15
dtype: float64

In [ ]:
# clearly, the datetimes and triptime are heavily correlated, so lets take one of the datetimes out and re-check!
col_list.remove("tpep_dropoff_datetime")

VIF((train), col_list)

VendorID                 4.911823
tpep_pickup_datetime     1.011502
passenger_count          1.023150
trip_distance            2.040065
RatecodeID               1.038791
store_and_fwd_flag       1.029657
PULocationID             1.033992
DOLocationID             1.030029
payment_type             1.463562
fare_amount              1.010319
extra                    4.975927
mta_tax                  1.194008
tip_amount               2.025603
tolls_amount             1.662999
improvement_surcharge    1.036199
congestion_surcharge     1.325501
trip_time                1.026757
dtype: float64

In [ ]:
# we can also remove fare amount, as it has the highest VIF and is likely most indicative of total_amount
col_list.remove("fare_amount")

VIF((train), col_list)

VendorID                 4.909533
tpep_pickup_datetime     1.011431
passenger_count          1.023140
trip_distance            2.039348
RatecodeID               1.038769
store_and_fwd_flag       1.029649
PULocationID             1.033935
DOLocationID             1.030001
payment_type             1.462846
extra                    4.973986
mta_tax                  1.184769
tip_amount               2.025598
tolls_amount             1.661960
improvement_surcharge    1.035897
congestion_surcharge     1.325469
trip_time                1.026749
dtype: float64

In [ ]:
# finally, we can remove "extra" from the cost as it correlates heavily with the VendorID
col_list.remove("extra")

VIF((train), col_list)
# here is our final list of relevant columns to test!

VendorID                 1.051289
tpep_pickup_datetime     1.010710
passenger_count          1.022888
trip_distance            2.003042
RatecodeID               1.036427
store_and_fwd_flag       1.029524
PULocationID             1.033375
DOLocationID             1.029940
payment_type             1.462846
mta_tax                  1.182672
tip_amount               2.023770
tolls_amount             1.661399
improvement_surcharge    1.035782
congestion_surcharge     1.178622
trip_time                1.026707
dtype: float64

In [ ]:
train.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,trip_time
21796,2.0,2.459499e+06,2.459499e+06,1.0,4.33,1.0,0,161,232,1.0,14.5,0.5,0.5,2.00,0.0,0.3,20.30,2.5,711
13790,2.0,2.459416e+06,2.459416e+06,1.0,2.30,1.0,0,236,230,1.0,13.0,1.0,0.5,3.46,0.0,0.3,20.76,2.5,1184
3131,1.0,2.459283e+06,2.459283e+06,1.0,1.00,1.0,0,239,238,1.0,5.5,2.5,0.5,1.00,0.0,0.3,9.80,2.5,243
16725,1.0,2.459448e+06,2.459448e+06,1.0,1.70,1.0,0,236,162,1.0,8.0,2.5,0.5,1.00,0.0,0.3,12.30,2.5,467
2311,1.0,2.459269e+06,2.459269e+06,1.0,0.70,1.0,0,186,90,1.0,4.5,3.5,0.5,1.75,0.0,0.3,10.55,2.5,169


In [ ]:
#OLS MODEL

#getting the names of the columns in the dataset
cols = "VendorID +" + " + ".join([i for i in col_list[1:]])
cols

# Testing out basic model with all variables
ols = smf.ols(formula='total_amount ~ ' + cols, 
                 data=train)
model1 =ols.fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:           total_amount   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     13.51
Date:                Mon, 08 May 2023   Prob (F-statistic):           9.76e-35
Time:                        23:48:07   Log-Likelihood:            -1.9120e+05
No. Observations:               20473   AIC:                         3.824e+05
Df Residuals:                   20457   BIC:                         3.826e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept               6.14e+

As we can see, VendorID, passenger_count, store_and_fwd_flag, PULocationID, and congestion_surcharge are not statistically significant with a cutoff of 0.05, as their p-values are larger than 0.05. This model is quite good on the training data: it has an r^2 value of 0.88.

In [ ]:
# test on the test set!

# compute out-of-sample R-squared using the test set
def OSR2(model, df_train, df_test, dependent_var):   
    y_test = df_test[dependent_var]
    y_pred = model.predict(df_test)
    SSE = np.sum((y_test - y_pred)**2)
    SST = np.sum((y_test - np.mean(df_train[dependent_var]))**2)    
    return 1 - SSE/SST

In [ ]:
OSR2(model1, train, test, "total_amount")

-129.53393427032074

In [ ]:
col_list

['VendorID',
 'tpep_pickup_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'congestion_surcharge',
 'trip_time']

In [ ]:
# removed columns based on significance values
cols_new = "tpep_pickup_datetime + trip_distance + RatecodeID + payment_type + mta_tax + tip_amount + tolls_amount + congestion_surcharge + trip_time"

In [ ]:
# Testing out OLS model with significant variables
ols_2 = smf.ols(formula='total_amount ~ ' + cols_new, 
                 data=train)
model2 =ols_2.fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:           total_amount   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     21.39
Date:                Mon, 08 May 2023   Prob (F-statistic):           1.84e-36
Time:                        23:48:32   Log-Likelihood:            -1.9121e+05
No. Observations:               20473   AIC:                         3.824e+05
Df Residuals:                   20463   BIC:                         3.825e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept             6.276e+05 

In [ ]:
OSR2(model2, train, test, "total_amount")

-119.17287561400515

In [ ]:
# Random forest model 

from sklearn.ensemble import RandomForestRegressor

model3 = RandomForestRegressor(n_estimators=150, max_depth=7, random_state=42)
model3.fit(train.drop('total_amount', axis=1), train[['total_amount']])

<ipython-input-33-fe1fcd0f8734>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model3.fit(train.drop('total_amount', axis=1), train[['total_amount']])


RandomForestRegressor(max_depth=7, n_estimators=150, random_state=42)

In [ ]:
y_pred = model3.predict(test.drop('total_amount', axis=1))
y_test = test['total_amount']

def OSR2_new(model, df_train, y_test, y_pred, dependent_var):   
    SSE = np.sum((y_test - y_pred)**2)
    SST = np.sum((y_test - np.mean(df_train[dependent_var]))**2)    
    return 1 - SSE/SST

OSR2_new(model3, train, y_test, y_pred, "total_amount")

0.9933620570790321

In [ ]:
from sklearn.metrics import r2_score
r2 = model3.score(test.drop('total_amount', axis=1), test[['total_amount']])
print("R-squared:", r2)

R-squared: 0.9821404263635211


In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print("Mean squared error:", mse)

accuracy = model3.score(test.drop('total_amount', axis=1), test[['total_amount']])
print("Accuracy:", accuracy)


Mean squared error: 4.000996016569917
Accuracy: 0.9821404263635211


In [ ]:
y_pred

array([ 8.96401481, 13.11582839, 13.36422202, ..., 14.80871494,
       14.64061564, 16.56775946])

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(train.drop('total_amount', axis=1))
X_test_pca = pca.transform(test.drop('total_amount', axis=1))

model4 = RandomForestRegressor(n_estimators=100, max_depth=6, random_state=42)
model4.fit(X_train_pca, train[['total_amount']])


<ipython-input-38-892330fbd617>:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model4.fit(X_train_pca, train[['total_amount']])


RandomForestRegressor(max_depth=6, random_state=42)

In [ ]:
y_pred = model4.predict(X_test_pca)
y_test = test['total_amount']

r2 = model4.score(X_test_pca, test[['total_amount']])
print("R-squared:", r2)

mse = mean_squared_error(y_test, y_pred)
print("Mean squared error:", mse)

accuracy = model4.score(X_test_pca, test[['total_amount']])
print("Accuracy:", accuracy)


R-squared: 0.9588170941034201
Mean squared error: 9.22601209843188
Accuracy: 0.9588170941034201


Now onto the Modeling for the 2nd dataset with Uber/Lyft/Via data!

In [ ]:
fht_2021.drop("fht_company", inplace=True, axis=1)

In [ ]:
train_fht, test_fht = train_test_split(fht_2021,test_size=0.3, random_state = 42)

In [ ]:
# cleaning for modeling/VIF test!

# extracting columns as a list
col_list_fht = list(train_fht.columns)
col_list_fht.remove("total_fare")

# converting datetime to julian datetime
train_fht["pickup_datetime"] = train_fht["pickup_datetime"].apply(lambda x: x.to_julian_date())
train_fht["dropoff_datetime"] = train_fht["dropoff_datetime"].apply(lambda x: x.to_julian_date())
train_fht["request_datetime"] = train_fht["request_datetime"].apply(lambda x: x.to_julian_date())

test_fht["pickup_datetime"] = test_fht["pickup_datetime"].apply(lambda x: x.to_julian_date())
test_fht["dropoff_datetime"] = test_fht["dropoff_datetime"].apply(lambda x: x.to_julian_date())
test_fht["request_datetime"] = test_fht["request_datetime"].apply(lambda x: x.to_julian_date())

In [ ]:
# replace Y and N with 0s and 1s
train_fht.replace("Y", 1, inplace=True)
train_fht.replace("N", 0, inplace=True)

test_fht.replace("Y", 1, inplace=True)
test_fht.replace("N", 0, inplace=True)

train_fht.head(15)

,request_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,...,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag,total_fare,Lyft,Uber,Via
14144152,2.459576e+06,2.459576e+06,2.459576e+06,258,198,3.860,1127,15.99,0.0,0.48,...,0.00,0,0,,0,1,17.89,0,1,0
14098181,2.459576e+06,2.459576e+06,2.459576e+06,262,164,4.304,1199,18.74,0.0,0.56,...,14.82,0,0,0,0,0,23.71,1,0,0
11097906,2.459569e+06,2.459569e+06,2.459569e+06,163,42,6.090,1911,34.56,0.0,1.04,...,26.55,0,0,,0,0,49.13,0,1,0
12624104,2.459362e+06,2.459362e+06,2.459362e+06,219,265,4.050,969,18.78,0.0,0.56,...,15.72,0,0,,0,0,21.01,0,1,0
10760339,2.459539e+06,2.459539e+06,2.459539e+06,249,114,1.200,683,9.41,0.0,0.28,...,10.20,0,0,,0,0,16.28,0,1,0
13916644,2.459425e+06,2.459425e+06,2.459425e+06,161,265,34.060,3020,99.60,0.0,2.99,...,70.43,0,0,,0,0,118.24,0,1,0
2765572,2.459223e+06,2.459223e+06,2.459223e+06,61,177,2.257,835,10.43,0.0,0.31,...,9.49,0,0,0,0,0,11.67,1,0,0
1019382,2.459522e+06,2.459522e+06,2.459522e+06,254,18,3.420,1075,18.05,0.0,0.54,...,13.00,0,0,,0,0,20.19,0,1,0
7949270,2.459383e+06,2.459383e+06,2.459383e+06,148,209,1.850,629,18.47,0.0,0.55,...,24.73,0,0,,0,0,23.41,0,1,0
8940850,2.459536e+06,2.459536e+06,2.459536e+06,68,246,1.210,334,7.91,0.0,0.24,...,6.82,0,0,,0,0,11.60,0,1,0


In [ ]:
# filling with 0s
train_fht.replace(" ", 0 , inplace=True)
test_fht.replace(" ", 0, inplace = True)

In [ ]:
# VIF score for all columns
VIF(train_fht, col_list_fht)

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1752: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1752: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


request_datetime        1.797537e+06
pickup_datetime         1.105838e+07
dropoff_datetime        1.538391e+09
PULocationID            1.044753e+00
DOLocationID            1.053258e+00
trip_miles              5.562729e+00
trip_time               4.065774e+02
base_passenger_fare     2.244333e+01
tolls                   2.974727e+00
bcf                     2.553520e+01
sales_tax               3.642561e+00
congestion_surcharge    1.181882e+00
airport_fee             1.235229e+00
tips                    1.139383e+00
driver_pay              1.187323e+01
shared_request_flag     1.944941e+00
shared_match_flag       1.472054e+00
access_a_ride_flag               NaN
wav_request_flag        1.025503e+00
wav_match_flag          1.057533e+00
Lyft                    4.503600e+15
Uber                             inf
Via                              inf
dtype: float64

In [ ]:
 # clearly, the datetimes are heavily correlated, so lets take out two of the datetimes and re-check! We will also remove Lyft, for multicollinearity,
 # along with access_a_ride_flag due to poor data input
col_list_fht.remove("pickup_datetime")
col_list_fht.remove("request_datetime")
col_list_fht.remove("access_a_ride_flag")
col_list_fht.remove("Lyft")


VIF(train_fht, col_list_fht)

dropoff_datetime         1.042920
PULocationID             1.044697
DOLocationID             1.053110
trip_miles               5.518402
trip_time                4.521287
base_passenger_fare     22.422521
tolls                    2.974592
bcf                     25.534476
sales_tax                3.629522
congestion_surcharge     1.175938
airport_fee              1.235170
tips                     1.139067
driver_pay              11.843709
shared_request_flag      1.942187
shared_match_flag        1.472046
wav_request_flag         1.024321
wav_match_flag           1.057251
Uber                     1.065624
Via                      1.790920
dtype: float64

In [ ]:
# we can remove the bcf, along with base_passenger_fare and driver_pay, in this case due to high VIFs. We can keep Uber / Lyft on because those variables
# are important in our case

col_list_fht.remove("bcf")
col_list_fht.remove("base_passenger_fare")
col_list_fht.remove("driver_pay")

VIF(train_fht, col_list_fht)
# lets use this for OLS now!

dropoff_datetime        1.035445
PULocationID            1.044350
DOLocationID            1.052263
trip_miles              3.652915
trip_time               3.247778
tolls                   1.544038
sales_tax               2.526082
congestion_surcharge    1.169887
airport_fee             1.222235
tips                    1.111142
shared_request_flag     1.918879
shared_match_flag       1.470828
wav_request_flag        1.022963
wav_match_flag          1.031264
Uber                    1.017656
Via                     1.465878
dtype: float64

In [ ]:
#OLS MODEL

#getting the names of the columns in the dataset
cols = "dropoff_datetime +" + " + ".join([i for i in col_list_fht[1:]])
cols

# Testing out basic model with all variables
ols = smf.ols(formula='total_fare ~ ' + cols, 
                 data=train_fht)
model1 =ols.fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:             total_fare   R-squared:                       0.910
Model:                            OLS   Adj. R-squared:                  0.910
Method:                 Least Squares   F-statistic:                 1.325e+04
Date:                Mon, 08 May 2023   Prob (F-statistic):               0.00
Time:                        23:49:32   Log-Likelihood:                -69300.
No. Observations:               21000   AIC:                         1.386e+05
Df Residuals:                   20983   BIC:                         1.388e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept            -1.163e+04 

In [ ]:
# removing non significant values
# wav_match_flag is very close to 0.05 p-value as well, so we will drop that
sig_fht = col_list_fht.copy()
sig_fht.remove("PULocationID")
sig_fht.remove("shared_match_flag")
sig_fht.remove("wav_match_flag")

# cleaning to a str
sig_fht = "dropoff_datetime +" + " + ".join([i for i in sig_fht[1:]])

# Testing out OLS model with significant variables
ols_2 = smf.ols(formula='total_fare ~ ' + sig_fht, 
                 data=train_fht)
model2 =ols_2.fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:             total_fare   R-squared:                       0.910
Model:                            OLS   Adj. R-squared:                  0.910
Method:                 Least Squares   F-statistic:                 1.630e+04
Date:                Mon, 08 May 2023   Prob (F-statistic):               0.00
Time:                        23:49:34   Log-Likelihood:                -69304.
No. Observations:               21000   AIC:                         1.386e+05
Df Residuals:                   20986   BIC:                         1.387e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept            -1.168e+04 

In [ ]:
# getting our model's OSR^2 on test set, along with MSE
y_test = test_fht["total_fare"]
y_pred = model2.predict(test_fht)

osr = OSR2(model2, train_fht, test_fht, "total_fare")

print("OSR^2:", osr)

mse = mean_squared_error(y_test, y_pred)
print("Mean squared error:", mse)

OSR^2: 0.9103960980888052
Mean squared error: 45.77252580429907


In [ ]:
model3 = RandomForestRegressor(n_estimators=150, max_depth=7, random_state=42)
model3.fit(train_fht.drop('total_fare', axis=1), train_fht[['total_fare']])

y_pred = model3.predict(test_fht.drop('total_fare', axis=1))
y_test = test_fht['total_fare']

r2 = OSR2_new(model3, train_fht, y_test, y_pred, "total_fare")
print("OSR-squared:", r2)

mse = mean_squared_error(y_test, y_pred)
print("Mean squared error:", mse)

accuracy = model3.score(test_fht.drop('total_fare', axis=1), test_fht[['total_fare']])
print("Accuracy:", accuracy)

<ipython-input-51-089457934173>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model3.fit(train_fht.drop('total_fare', axis=1), train_fht[['total_fare']])


OSR-squared: 0.9871004049908794
Mean squared error: 6.589523813429101
Accuracy: 0.9871000185035745


In [ ]:
pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(train_fht.drop('total_fare', axis=1))
X_test_pca = pca.transform(test_fht.drop('total_fare', axis=1))

model4 = RandomForestRegressor(n_estimators=100, max_depth=6, random_state=42)
model4.fit(X_train_pca, train_fht[['total_fare']])

y_pred = model4.predict(X_test_pca)
y_test = test_fht['total_fare']

r2 = OSR2_new(model4, train_fht, y_test, y_pred, "total_fare")
print("OSR-squared:", r2)

mse = mean_squared_error(y_test, y_pred)
print("Mean squared error:", mse)

r2 = model4.score(X_test_pca, test_fht[['total_fare']])
print("Accuracy:", r2)